# BioExplorer - CCFv3
![](../bioexplorer_ccfv3_banner.png)

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer, MovieMaker, Vector3
from tqdm import tqdm
import glob
import os

url = 'localhost:5000'
be = BioExplorer(url)
core = be.core_api()
mm = MovieMaker(be)

## Dataset

In [ ]:
species_folder = '/scratch/spinal_cord/3D_Mouse_SC_Atlas/Mouse_Spinal_Cord_3D_Atlas'
create_snapshot = False
transparency = 0.1

output_folder = '/scratch/images/atlas/'
os.makedirs(output_folder, exist_ok=True)

## Images

### Spinal cord

In [ ]:
def set_materials(model_id, color, opacity):
    colors = list()
    material_ids = be.get_material_ids(model_id)['ids']
    shading_modes = list()
    opacities = list()
    user_params = list()
    refraction_indices = list()
    specular_exponents = list()
    glossinesses = list()
    for _ in material_ids:
        colors.append(color)
        opacities.append(opacity)
        shading_modes.append(be.shading_mode.DIFFUSE)
        user_params.append(0.00005)
        refraction_indices.append(1.0)
        specular_exponents.append(50.0)
        glossinesses.append(0.1)
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        shading_modes=shading_modes, opacities=opacities, user_parameters=user_params,
        diffuse_colors=colors, specular_colors=colors, specular_exponents=specular_exponents,
        refraction_indices=refraction_indices, glossinesses=glossinesses)

In [ ]:
def create_image(folder_name, sample_folder, color):
    files = glob.glob(os.path.join(sample_folder, '*.obj'))
    model_bounds_list = list()
    group_model_ids = list()
    to_show = ['df', 'lvf']

    for file in files:
        base_name = os.path.basename(file)
        values = base_name.split('_')
        model_name = values[1].replace('.obj', '')
       
        if model_name not in to_show:
            continue

        model = core.add_model(name=folder_name + '_' + model_name, path=file)
        model_id = model['id']
        group_model_ids.append(model_id)
        model_bounds_list.append(model['bounds'])
        import time
        time.sleep(0.1)


In [ ]:
def load_data():
    import seaborn as sns
    be.reset_scene()
    sample_folders = glob.glob(os.path.join(species_folder, '*'))
    sample_folders.sort()

    sections = ['C', 'T', 'L', 'S',  'Co']
    section_colors = dict()
    palette = sns.color_palette('rainbow', len(sections))

    i = 0
    for section_name in sections:
        section_colors[section_name] = palette[i]
        i += 1

    for sample_folder in tqdm(sample_folders):
        values = sample_folder.split('/')
        folder_name = values[len(values) - 1]
        section_name = folder_name.split('_')[1].replace('vert', '')
        section_name = ''.join([i for i in section_name if not i.isdigit()])
        create_image(
            section_name,
            sample_folder,
            section_colors[section_name] # Colored by section
        )
        i += 1

load_data()

In [ ]:
model_ids = be.get_model_ids()['ids']
for i in range(68):
    model_id = model_ids[i]
    tf = {
        'rotation': [-0.5, -0.5, -0.5, 0.5],
        'rotation_center': [0.0, 0.0, 0.0],
        'scale': [1200.0, 1200.0, 1200.0],
        'translation': [10.95, 5.45, 4.75]
    }
    core.update_model(model_id, transformation=tf)


In [ ]:
color_offset = dict()
color_offset['C'] = [0.0, 0.0, 0.0]
color_offset['T'] = [0.0, -0.05, 0.05]
color_offset['L'] = [0.0, -0.1, 0.1]
color_offset['S'] = [0.0, -0.15, 0.15]
color_offset['Co'] = [0.0, -0.2, 0.2]
model_ids = be.get_model_ids()['ids']
for i in range(68):
    model_id = model_ids[i]
    model_name = be.get_model_name(model_id)['name']
    prefix = model_name.split('_')[0]
    color = [255.0/256.0, 155.0/256.0, 205.0/256.0]
    color[1] += color_offset[prefix][1] * 2
    color[2] += color_offset[prefix][2] * 2
    set_materials(model_id, color, 1.0)

### Brain

In [ ]:
population_name = 'ccfv3a'

In [ ]:
cerebellum_ids = [
       512,      1025,       519,      1033,       528,      1041,      1049,
      1056,      1064,      1073,      1091,       846,        91, 589508455,
       989,      1143,      1144,      1145,       645,       912,       920,
       928,       936,       944,     10672,     10673,     10674,     10675,
     10676,     10677,       951,     10680,     10679,     10678,     10681,
     10682,       957,     10683,     10687,     10688,     10689,     10690,
     10691,     10692,     10684,     10686,       968,       976,     10705,
     10706,     10707,     10708,     10709,     10710,     10711,     10712,
     10713,       984,     10714,     10715,     10716,     10717,     10718,
       992,     10720,     10721,     10722,     10719,     10723,     10724,
     10725,     10728,      1001,     10726,     10727,     10729,     10730,
     10731,     10733,     10732,     10734,     10737,     10736,     10735,
      1007,      1017,     10685
]

cerebellum_granular_layer_ids = [10675, 10678, 10708, 10711, 10684, 10720, 10723, 10690, 10726, 10705, 10735, 10687, 10681, 10729, 10672, 10732]
cerebellum_molecular_layer_ids =  [10677, 10680, 10710, 10713, 10686, 10722, 10725, 10692, 10728, 10707, 10737, 10689, 10683, 10731, 10674, 10734]


olfactory_bulb_ids = [507, 212, 228, 236, 244, 220]

medula_ids = [
      773,       781,      1039,       789,      1048,        45,      1069,
      560,       307,        53,       568,        61,       576,       69,
      839,      1098,        76,        77,       83,      1107,       852,
      859,       607,        96,       354, 589508451,       101,       106,
      112,       370,       372,       887,       379,       640,      386,
      642,       903,       135,       136,       651,       395,       653,
      143,       659,       661,       666,       154,       161,       674,
      169,       682,       938,       939,       429,       177,       691,
      437,       185,       955,       445,       701,       193,       963,
      711,       970,       203,       202,       206,       207,       720,
      209,       978,       217,       222,       225,       995,       230,
      235,       765, 
      1557651847
]

test_list = [
  256,  260,  517, 1034,  267,  268, 1042,  276,  788,  535, 1050,  284, 1059,  291,
  297, 1067,  814,  306, 1075,  566, 1082,  584,  589,  592,  597,  605,  360,  619,
 1139, 1140, 1141, 1142,  631,  376,  639,  383,  646,  647,  392,  655,  400,  151,
  152,  663,  408,  159,  160,  416,  167,  168,  424,  175,  183,  698,  188,  191,
  192,  961,  196,  199,  200,  204,  208,  216,  224,  232,  496,  240,  248, 
  # 507,  212,  228,  236,  244,  220
]


fibers_ids = [960, 1000, 1009, 396, 109]

tracts_ids = [619, 392, 260, 268, 1139, 292, 628, 66, 75, 58, 651, 659, 666, 674, 682, 691, 2316153360, 1009, 21, 665, 459, 876, 125, 705, 794, 237, 932, 871, 29, 389, 245, 627, 960, 85, 866, 553, 499, 490, 404, 410, 373, 784, 1036, 1012, 1003, 994, 1019, 1028, 2718688460, 102, 109, 863, 221, 1428498274, 855, 205, 213, 428, 405, 753, 690, 681, 653, 2500193001]

seb_ids = [
  256, 260, 517, 1034, 267, 268, 1042, 276, 788, 535, 1050, 284, 1059, 291, 1209357605,
  297, 1067, 814, 306, 1075, 566, 1668688439, 1082, 584, 1024543562, 589, 592, 597,
  1992072790, 605, 2358040414, 1860102496, 360, 619, 1953921139, 1139, 1375046773, 1140,
  631, 376, 1141, 1142, 383, 639, 646, 647, 392, 655, 400, 151, 152, 663, 1203939479,
  408, 159, 160, 416, 167, 168, 424, 175, 183, 3389528505, 698, 188, 191, 192, 961,
  196, 199, 200, 204, 208, 212, 216, 220, 224, 228, 232, 236, 1466095084, 496, 240,
  244, 248, 507, 2561915647
]

In [ ]:
cerrebelum_filter = str(cerebellum_ids).replace('[','').replace(']','')
cerebellum_granular_layer_filter = str(cerebellum_granular_layer_ids).replace('[','').replace(']','')
cerebellum_molecular_layer_filter = str(cerebellum_molecular_layer_ids).replace('[','').replace(']','')
olfactory_bulb_filter = str(olfactory_bulb_ids).replace('[','').replace(']','')
medula_filter = str(medula_ids).replace('[','').replace(']','')
fibers_filter = str(fibers_ids).replace('[','').replace(']','')

In [ ]:
import json
import os
atlas_data_folder = '/home/favreau/medias/atlas'
data_folder = os.path.join(atlas_data_folder, 'mouse', 'CCFv2', 'barrel_split')

region_file_name = 'hierarchy_ccfv2_l23split_barrelsplit.json'
# f = open(os.path.join(data_folder, region_file_name))


f = open('brain_regions.json')
obj = json.load(f)

regions_with_no_children = list()
region_colors=dict()
def node_color(node, level):
    node_id = node['id']
    color = node['color_hex_triplet']
    region_colors[node_id] = color
    children = node['children']
    if not children:
        regions_with_no_children.append(node_id)
    for child in children:
        node_color(child, level + 1)

for node in obj['msg']:
    node_color(node, 0)

def hex_to_rgb(value):
    value = value.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))

In [ ]:
atlas_assembly_name = 'cerrebelum'
be.remove_assembly(atlas_assembly_name)
atlas_assembly = be.add_assembly(atlas_assembly_name)
atlas_model = be.add_atlas(
    assembly_name=atlas_assembly_name,
    population_name=population_name,
    load_cells=False, load_meshes=True,
    region_sql_filter='guid IN (%s)' % cerrebelum_filter,
    mesh_scale=Vector3(1, 1, 1)
)

In [ ]:
atlas_assembly_name = 'cerebellum_granular_layer'
be.remove_assembly(atlas_assembly_name)
atlas_assembly = be.add_assembly(atlas_assembly_name)
atlas_model = be.add_atlas(
    assembly_name=atlas_assembly_name,
    population_name=population_name,
    load_cells=False, load_meshes=True,
    region_sql_filter='guid IN (%s)' % cerebellum_granular_layer_filter,
    mesh_scale=Vector3(1, 1, 1)
)

In [ ]:
atlas_assembly_name = 'cerebellum_molecular_layer'
be.remove_assembly(atlas_assembly_name)
atlas_assembly = be.add_assembly(atlas_assembly_name)
atlas_model = be.add_atlas(
    assembly_name=atlas_assembly_name,
    population_name=population_name,
    load_cells=False, load_meshes=True,
    region_sql_filter='guid IN (%s)' % cerebellum_molecular_layer_filter,
    mesh_scale=Vector3(1, 1, 1)
)

In [ ]:
atlas_assembly_name = 'cerrebelum'
be.remove_assembly(atlas_assembly_name)
atlas_assembly = be.add_assembly(atlas_assembly_name)
atlas_model = be.add_atlas(
    assembly_name=atlas_assembly_name,
    population_name=population_name,
    load_cells=False, load_meshes=True,
    region_sql_filter='guid IN (%s) AND guid NOT IN (%s) AND guid NOT IN (%s)' % (cerrebelum_filter, cerebellum_granular_layer_filter, cerebellum_molecular_layer_filter),
    mesh_scale=Vector3(1, 1, 1)
)

In [ ]:
atlas_assembly_name = 'olfactory_bulb'
be.remove_assembly(atlas_assembly_name)
atlas_assembly = be.add_assembly(atlas_assembly_name)
atlas_model = be.add_atlas(
    assembly_name=atlas_assembly_name,
    population_name=population_name,
    load_cells=False, load_meshes=True,
    region_sql_filter='guid IN (%s)' % olfactory_bulb_filter,
    mesh_scale=Vector3(1, 1, 1)
)

In [ ]:
atlas_assembly_name = 'medula'
be.remove_assembly(atlas_assembly_name)
atlas_assembly = be.add_assembly(atlas_assembly_name)
atlas_model = be.add_atlas(
    assembly_name=atlas_assembly_name,
    population_name=population_name,
    load_cells=False, load_meshes=True,
    region_sql_filter='guid IN (%s)' % medula_filter,
    mesh_scale=Vector3(1, 1, 1)
)

In [ ]:
atlas_assembly_name = 'other_regions'
be.remove_assembly(atlas_assembly_name)
atlas_assembly = be.add_assembly(atlas_assembly_name)
all_filter = olfactory_bulb_ids + medula_ids + cerebellum_ids + cerebellum_granular_layer_ids + cerebellum_molecular_layer_ids
all_filter = str(all_filter).replace('[', '').replace(']', '')
atlas_model = be.add_atlas(
    assembly_name=atlas_assembly_name,
    population_name=population_name,
    load_cells=False, load_meshes=True,
    region_sql_filter='level in (6)  AND guid NOT IN (1024543562) AND guid NOT IN (%s)' % all_filter,
    mesh_scale=Vector3(1, 1, 1)
)

In [ ]:
model_ids = be.get_model_ids()['ids'][-6:]

shading_mode = be.shading_mode.DIFFUSE
exponent = 50.0

for model_id in tqdm(model_ids):
    material_ids = be.get_material_ids(model_id)['ids'][:-1]
    palette = list()
    opacities = list()
    shading_modes = list()
    specular_exponents = list()
    user_params = list()
    refraction_indices = list()
    glossinesses = list()
    reflection_indices = list()
    for material_id in material_ids:
        c = [255, 0, 0]
        m_id = material_id
        if m_id in region_colors:
            id = region_colors[m_id]
            c = hex_to_rgb(id)
        alpha = 1.0
        if (m_id in cerebellum_ids and m_id not in cerebellum_granular_layer_ids and m_id not in cerebellum_molecular_layer_ids) or m_id in olfactory_bulb_ids or m_id in medula_ids:
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            alpha = 1.0
            user_params.append(0.00001)
            glossinesses.append(0.1)
            specular_exponents.append(exponent)
            reflection_indices.append(0.0)
        elif m_id in cerebellum_granular_layer_ids:
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            alpha = 0.5
            user_params.append(0.00001)
            glossinesses.append(0.1)
            specular_exponents.append(exponent)
            reflection_indices.append(0.0)
        elif m_id in cerebellum_molecular_layer_ids:
            opacities.append(1.0)
            shading_modes.append(shading_mode)
            alpha = 1.0
            user_params.append(0.00001)
            glossinesses.append(0.1)
            specular_exponents.append(exponent)
            reflection_indices.append(0.0)
        else:
            opacities.append(0.3)
            alpha = 2.0
            # opacities.append(1.0)
            # alpha = 0.5
            shading_modes.append(shading_mode)
            user_params.append(3.0)
            glossinesses.append(0.5)
            specular_exponents.append(50.0)
            reflection_indices.append(0.0)
        refraction_indices.append(1.0)
        palette.append([alpha * c[0] / 255.0, alpha * c[1] / 255.0, alpha * c[2] / 255.0])
        
    be.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        shading_modes=shading_modes, user_parameters=user_params,
        opacities=opacities, specular_exponents=specular_exponents,
        reflection_indices=reflection_indices,
        refraction_indices=refraction_indices, glossinesses=glossinesses,
        diffuse_colors=palette, specular_colors=palette)

In [ ]:
status = core.set_renderer(
    head_light=False,
    current='advanced',
    background_color=[11.0/256.0, 67/256.0, 14/256.0],
    subsampling=4, max_accum_frames=512)
params = core.AdvancedRendererParams()
params.gi_ray_length = 200.0
params.shadow_intensity = 0.5
params.soft_shadow_strength = 0.25
params.main_exposure = 1.25
params.max_ray_depth = 30
params.epsilon_multiplier = 5.0
params.use_hardware_randomizer = False
params.show_background = False
status = core.set_renderer_params(params)

In [ ]:
status = core.clear_lights()
core.add_light_directional(
    angularDiameter=45, color=(1,1,1), direction=(-0.5, 0.5, 0.2),
    intensity=0.8, is_visible=False)

### Floor

In [ ]:
if False:

    from bioexplorer import Vector3
    status = be.add_box(
        name='Floor',
        bottom_left_corner=Vector3(-500000, 8000, -500000), top_right_corner=Vector3(500000, 8200, 500000))

    model_id = be.get_model_ids()['ids'][:-1][0]
    set_materials(model_id, [1, 1, 1], 1.0)

    status = core.set_renderer(
        head_light=False,
        current='advanced',
        background_color=[11.0/256.0, 67/256.0, 14/256.0],
        subsampling=4, max_accum_frames=512)
    params = core.AdvancedRendererParams()
    params.gi_ray_length = 5000.0
    params.shadow_intensity = 0.8
    params.soft_shadow_strength = 1.0
    params.main_exposure = 1.5
    params.max_ray_depth = 30
    params.epsilon_multiplier = 4.0
    params.use_hardware_randomizer = False
    params.show_background = False
    status = core.set_renderer_params(params)    

## Snapshots

### 3D view

In [ ]:
output_folder = '/scratch/images/atlas'
k = 4
spp = 256
image_size = [k*960, k*540]

In [ ]:
core.set_camera(
    current='perspective',
    orientation=[0.8541502729779541, 0.0622479784707429, -0.3074297814156883, 0.41477636124385714],
    position=[624.61022325259, -16843.11224754929, -9430.324215398294],
    target=[14712.6758305083, 5375.456994337424, 4459.917593163967],
)

In [ ]:
mm.create_snapshot(
    renderer='advanced',
    path=output_folder, show_progress=True,
    base_name='ccfv2_with_spinal_cord_figure_%dx%d_%dspp_v1' % (image_size[0], image_size[1], spp),
    size=image_size, samples_per_pixel=spp)

### Sagital view

In [ ]:
status = core.set_camera(
    current='orthographic',
    orientation=[-1.0, 0.0, 0.0, 0.0],
    position=[25277.44517898313, 5087.062515237374, -41859.262825475045],
    target=[25277.44517898312, 5087.062515237374, 5.777407429365563e-11],
)
params = core.OrthographicCameraParams()
params.height = 55000
status = core.set_camera_params(params)

In [ ]:
mm.create_snapshot(
    renderer='advanced',
    path=output_folder, show_progress=True,
    base_name='ccfv2_with_spinal_cord_figure_%dx%d_%dspp_v2' % (image_size[0], image_size[1], spp),
    size=image_size, samples_per_pixel=spp)

### Coronal view

In [ ]:
status = core.set_camera(
    current='orthographic',
    orientation=[0.7071067811865475, 0.0, 0.0, 0.7071067811865476],
    position=[25246.78033755872, -21000.118117022015, 5290.873373618535],
    target=[25246.78033755872, 3717.4600460529327, 5290.8733736185295],
)
params = core.OrthographicCameraParams()
params.height = 55000
status = core.set_camera_params(params)

In [ ]:
mm.create_snapshot(
    renderer='advanced',
    path=output_folder, show_progress=True,
    base_name='ccfv2_with_spinal_cord_figure_%dx%d_%dspp_v3' % (image_size[0], image_size[1], spp),
    size=image_size, samples_per_pixel=spp)

### Closeup on spinal cord junction

In [ ]:
core.set_camera(
    current='perspective',
    orientation=[0.9026660109835698, -0.12968665364956194, 0.3442263848492494, 0.22334645834175407],
    position=[21663.61582469972, -1876.340583461724, -5053.494020929131],
    target=[11437.838731368503, 7060.760982298247, 6982.146512926149]
)

In [ ]:
mm.create_snapshot(
    renderer='advanced',
    path=output_folder, show_progress=True,
    base_name='ccfv2_with_spinal_cord_figure_%dx%d_%dspp_v4' % (image_size[0], image_size[1], spp),
    size=image_size, samples_per_pixel=spp)

## Debug

In [ ]:
import ipywidgets as widgets

class TreeWidget(widgets.VBox):
    def __init__(self, json_data):
        super().__init__()
        self._tree = self._build_tree(json_data)
        self.children = [self._tree]
        
    def _build_tree(self, data):
        tree = widgets.VBox()
        for item in data:
            label = widgets.Label(f"{item['id']}: {item['acronym']}")
            if 'children' in item and item['children']:
                children_tree = self._build_tree(item['children'])
                tree.children += (widgets.HBox([label, children_tree]),)
            else:
                tree.children += (label,)
        return tree

f = open(os.path.join(data_folder, region_file_name))
json_data = json.load(f)['msg']
tree = TreeWidget(json_data)
tree